In [8]:
import pandas as pd
import numpy as np
import pickle
from networkx import generators as gr
import networkx as nx
from pandasql import sqldf


In [3]:
role_list = {}
def test2(x):
    name = x['Name2'].unique()[0]
    if name in role_list:
        np.append(role_list[name],x['T2'].unique())
        role_list[name] = np.unique(role_list[name])
    else:
        role_list[name] = x['T2'].unique()
                               
def test(x):
        role_list[x['Name1'].unique()[0]] = x['T1'].unique()
    
def create_role_list(df):
    df.groupby('Name1').apply(test)
    df.groupby('Name2').apply(test2)

In [4]:
def add_centrality(G):
    
    Dict = nx.algorithms.centrality.degree_centrality(G)
    Min = min(Dict.values())
    Max = max(Dict.values())
    rmin,rmax = 20,40

    for u  in G.nodes():
        percent = (Dict[u] - Min) / (Max - Min)
        G.nodes[u]['centrality'] = percent * (rmax - rmin) + rmin  
    

In [5]:
def add_roles(G,role_list):
    for name,role in role_list.items():
        G.nodes[name]['Role'] = role

In [45]:
def update_graph_obj():
    
    df = pd.read_csv('UPDATED_SNA_data.csv',encoding = "ISO-8859-1",dtype={'IncidentNumber':object})
    df['InvolvementType'] = df['InvolvementType'].str.upper()
    df['InvolvementType']=df['InvolvementType'].map({'SUSPECT':0,'VICTIM':1,'WITNESS/OTHER':2})
    df['FullName'] = df['FullName'].str.strip()
    q = "SELECT distinct d1.Identifier,d1.FullName as Name1,d2.FullName as Name2,d1.InvolvementType as T1,d2.InvolvementType as T2 FROM df d1, df d2 WHERE d1.Identifier = d2.Identifier and d1.FullName > d2.FullName;"
    q2 = 'SELECT d1.Identifier from ndf d1 group by d1.Identifier having count(*) <= 190;'
    ndf = sqldf(q, locals())
    df2 = sqldf(q2, locals())
    df3 = ndf.join(df2.set_index('Identifier'),how='inner', on = "Identifier", lsuffix = "_df2",rsuffix = '_ndf')
    
    global role_list
    role_list = {}
    print('Adding Roles..')
    create_role_list(df3)
    #df = drop_roles(role, df, central_name)
    names = np.concatenate((df3.Name1.values,df3.Name2.values))
    names = sorted(list(set(names)))
    G = nx.from_pandas_edgelist(df3, 'Name1', 'Name2', True)
    add_roles(G,role_list)
    add_centrality(G)
    print('Saving to disk..')
    pickle.dump(G, open( "net.dat", "wb" ))
    pickle.dump(names, open( "names.dat", "wb" ))
    print('Done')
    return G



In [46]:
G = update_graph_obj()

Adding Roles..
Saving to disk..
Done
